In [1]:
import pandas as pd
import sqlite3

In [2]:
conn = sqlite3.Connection("./data/im.db")

# Creating dataframes from im.db.move_basics and im.db.movie_ratings



In [4]:
# Query to pull relevant data from im.db

# I think there's probably a better/more efficient query that would pull everything we need but I was getting stuck.
# Instead I decided to do two queries and merge the resulting dataframes.

q2 = """
SELECT
    movie_id,
    primary_title,
    start_year,
    runtime_minutes,
    genres,
    directors.person_id AS director_id,
    writers.person_id AS writer_id,  
    movie_ratings.averagerating,
    movie_ratings.numvotes
    
FROM
    movie_basics
    JOIN
        movie_ratings
            USING(movie_id)
    JOIN
        directors
            USING(movie_id)
    JOIN
        writers
            USING(movie_id)
    
WHERE
    start_year >= 2010 AND
    start_year <= 2022


GROUP BY
    movie_basics.movie_id
;
"""
q2_result = pd.read_sql(q2, conn)
q2_result

,movie_id,primary_title,start_year,runtime_minutes,genres,director_id,writer_id,averagerating,numvotes
0,tt0063540,Sunghursh,2013,175.0,"Action,Crime,Drama",nm0712540,nm0023551,7.0,77
1,tt0069049,The Other Side of the Wind,2018,122.0,Drama,nm0000080,nm0000080,6.9,4517
2,tt0069204,Sabse Bada Sukh,2018,NaN,"Comedy,Drama",nm0611531,nm0347899,6.1,13
3,tt0100275,The Wandering Soap Opera,2017,80.0,"Comedy,Drama,Fantasy",nm0749914,nm0749914,6.5,119
4,tt0137204,Joe Finds Grace,2017,83.0,"Adventure,Animation,Comedy",nm0365480,nm0365480,8.1,263
...,...,...,...,...,...,...,...,...,...
63083,tt9911774,Padmavyuhathile Abhimanyu,2019,130.0,Drama,nm10536451,nm10536451,8.4,365
63084,tt9913084,Diabolik sono io,2019,75.0,Documentary,nm0812850,nm0812850,6.2,6
63085,tt9914286,Sokagin Çocuklari,2019,98.0,"Drama,Family",nm4394529,nm1902682,8.7,136
63086,tt9914642,Albatross,2017,NaN,Documentary,nm5300859,nm5300859,8.5,8


   - Movie Title
   - Genre(s)
   - Release Year
   - Rating (R/PG-13/etc)
   - Total Gross
   - Production Budget
   - Associated Actors/Star/TBD
   - Writer
   - Director
   - Run Time
   - IMDB Rating
   - The Movie Database Popularity (still need to understand what this is)
   - The Movie Database Vote Count
   
   - season of release
   - inflation relative to budget/gross -- pull data & correlate with release year
   - franchise?

In [5]:
# query to create a mapping table so that we can replace director/writer ID with real names.

q3 = """
SELECT
    person_id,
    primary_name
        
FROM
    persons
   
    
GROUP BY
    person_id
;
"""
q3_result = pd.read_sql(q3, conn)
q3_result

,person_id,primary_name
0,nm0000002,Lauren Bacall
1,nm0000003,Brigitte Bardot
2,nm0000005,Ingmar Bergman
3,nm0000006,Ingrid Bergman
4,nm0000007,Humphrey Bogart
...,...,...
606643,nm9993494,Amjad Ali
606644,nm9993573,Lakisha Louissaint
606645,nm9993616,Ryan Mac Lennan
606646,nm9993650,Marcin Balcerak


In [6]:
q4 = """
SELECT
    known_for.person_id
    , persons.primary_name
    , group_concat(known_for.movie_id)    
    , group_concat(movie_basics.primary_title)
    
FROM
    known_for

    JOIN
        persons
            USING(person_id)
    JOIN
        movie_basics
            USING(movie_id)

GROUP BY
    persons.primary_name

;
"""
known_for = pd.read_sql(q4, conn)
known_for.head(25)

,person_id,primary_name,group_concat(known_for.movie_id),group_concat(movie_basics.primary_title)
0,nm6950093,!'aru Ikhuisi Piet Berendse,tt4245110,Origins
1,nm6750303,$hutter,"tt4049298,tt1398426,tt2080374,tt1741273","NVTION: The Star Nation Rapumentary,Straight O..."
2,nm2764095,'Atlas' Ramachandran,"tt3433162,tt2333760","Holidays,Three Char Sau Bees"
3,nm5169037,'Atolomake Helu,tt1966622,Tongan Ark
4,nm5420107,'Bad' Chad Broussard,tt1703232,Fightville
5,nm3961282,'Big' James Wroblewski,tt2215017,Extinction Event
6,nm6138665,'Brukup' George Adams,tt3115428,Lords of BSV
7,nm9141318,'Cicak' Zelimir Altarac,tt7154990,Zarobljeno vreme
8,nm4963520,'Country' Winston Marshall,"tt2948356,tt2622620,tt2073086","Zootopia,Bella Vita,Still Mine"
9,nm6246859,'Cowboy' Matt Hopewell,tt2338351,"Trash, a Bucky Jones Video"


In [7]:
# merge primary_name from the persons table onto our dataframe where director_id matches person_id
directors = q2_result.merge(q3_result, how='left', left_on='director_id', right_on='person_id')

In [8]:
# rename primary_name to director_name
directors = directors.rename({"primary_name":"director_name"}, axis="columns")

In [9]:
# merge primary_name from the persons table onto our dataframe where writer_id matches person_id
writers = directors.merge(q3_result, how='left', left_on='writer_id', right_on='person_id')

In [10]:
# rename primary_name to director_name
df = writers.rename({"primary_name":"writer_name"}, axis="columns")

In [11]:
# drop extraneous columns
df = df.drop(columns=['director_id', 'writer_id', 'person_id_x', 'person_id_y'])

In [25]:
# shape matches the length of our original table.
print(df.shape) 
df.head()

(63088, 9)


,movie_id,primary_title,start_year,runtime_minutes,genres,averagerating,numvotes,director_name,writer_name
0,tt0063540,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77,Harnam Singh Rawail,Abrar Alvi
1,tt0069049,The Other Side of the Wind,2018,122.0,Drama,6.9,4517,Orson Welles,Orson Welles
2,tt0069204,Sabse Bada Sukh,2018,NaN,"Comedy,Drama",6.1,13,Hrishikesh Mukherjee,Gulzar
3,tt0100275,The Wandering Soap Opera,2017,80.0,"Comedy,Drama,Fantasy",6.5,119,Raoul Ruiz,Raoul Ruiz
4,tt0137204,Joe Finds Grace,2017,83.0,"Adventure,Animation,Comedy",8.1,263,Anthony Harrison,Anthony Harrison


In [13]:
# some missing values here that we can either deal with or try to fill in from the other sources.
df.isna().sum()

movie_id              0
primary_title         0
start_year            0
runtime_minutes    5789
genres              471
averagerating         0
numvotes              0
director_name         1
writer_name           0
dtype: int64

In [14]:
conn.close()

In [36]:
df.duplicated(subset='primary_title').value_counts()

False    59813
True      3275
dtype: int64

<class 'pandas.core.frame.DataFrame'>
Int64Index: 63088 entries, 0 to 63087
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   movie_id         63088 non-null  object 
 1   primary_title    63088 non-null  object 
 2   start_year       63088 non-null  int64  
 3   runtime_minutes  57299 non-null  float64
 4   genres           62617 non-null  object 
 5   averagerating    63088 non-null  float64
 6   numvotes         63088 non-null  int64  
 7   director_name    63087 non-null  object 
 8   writer_name      63088 non-null  object 
dtypes: float64(2), int64(2), object(5)
memory usage: 4.8+ MB


In [37]:
df[df['primary_title'] == 'The Overnight']

,movie_id,primary_title,start_year,runtime_minutes,genres,averagerating,numvotes,director_name,writer_name
25,tt0326592,The Overnight,2010,88.0,None,7.5,24,Jed I. Goodman,Kacey Arnold
35529,tt3844362,The Overnight,2015,79.0,"Comedy,Mystery",6.1,14828,Patrick Brice,Patrick Brice


In [15]:
df_budget = pd.read_csv('./data/tn.movie_budgets.csv', index_col=0)

In [55]:
# convert release_date to a release_year column
df_budget['release_year'] = df_budget['release_date'].apply(lambda x: int(x[-4:]))

In [56]:
df_budget.head()

,release_date,movie,production_budget,domestic_gross,worldwide_gross,release_year
id,,,,,,
1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279",2009
2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875",2011
3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350",2019
4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963",2015
5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747",2017


In [57]:
# create simple budget df to merge with main df
df_budget_simple = df_budget[['movie', 'production_budget', 'worldwide_gross', 'release_year']]

In [58]:
# merge simple budget df to main df on movie name & year
df2 = df.merge(df_budget_simple, how='inner', left_on=['primary_title', 'start_year'], right_on=['movie', 'release_year'])

In [59]:
df2.isna().sum()

movie_id             0
primary_title        0
start_year           0
runtime_minutes      6
genres               1
averagerating        0
numvotes             0
director_name        0
writer_name          0
movie                0
production_budget    0
worldwide_gross      0
release_year         0
dtype: int64

In [60]:
df2.shape

(1479, 13)

In [80]:
# create dataframe containing records where inner join created primary_title == duplicated
df_dup = df2.loc[df2['primary_title'].duplicated(keep=False) == True]

In [81]:
df_dup.sort_values(by='primary_title').head(32)

,movie_id,primary_title,start_year,runtime_minutes,genres,averagerating,numvotes,director_name,writer_name,movie,production_budget,worldwide_gross,release_year
488,tt1554091,A Better Life,2011,98.0,"Drama,Romance",7.2,14602,Chris Weitz,Roger L. Simon,A Better Life,"$10,000,000","$1,884,251",2011
489,tt2027265,A Better Life,2011,110.0,Drama,6.6,1519,Cédric Kahn,Cédric Kahn,A Better Life,"$10,000,000","$1,884,251",2011
533,tt2447982,Abduction,2011,84.0,"Horror,Thriller",5.2,10,Michael Atwill,Shane Rodgers,Abduction,"$35,000,000","$86,648,359",2011
532,tt1600195,Abduction,2011,106.0,"Action,Mystery,Thriller",5.1,72552,John Singleton,Shawn Christensen,Abduction,"$35,000,000","$86,648,359",2011
703,tt2404163,After,2012,83.0,Romance,6.3,34,Géraldine Maillet,Géraldine Maillet,After,"$650,000",$0,2012
702,tt1799508,After,2012,90.0,"Drama,Fantasy,Mystery",5.5,4881,Ryan Smith,Jason Parish,After,"$650,000",$0,2012
1247,tt3802576,Brothers,2015,156.0,"Action,Drama,Sport",6.7,10689,Karan Malhotra,Cliff Dorfman,Brothers,"$13,000,000","$17,856,688",2015
1248,tt4952376,Brothers,2015,68.0,Documentary,7.5,61,Wojciech Staron,Wojciech Staron,Brothers,"$13,000,000","$17,856,688",2015
172,tt1126591,Burlesque,2010,119.0,"Drama,Music,Musical",6.4,71021,Steve Antin,Steve Antin,Burlesque,"$55,000,000","$90,552,675",2010
173,tt1586713,Burlesque,2010,NaN,Drama,7.0,45,Dominic Deacon,Dominic Deacon,Burlesque,"$55,000,000","$90,552,675",2010


# ** EVERYTHING BELOW IS NOT USED--just converted things to dataframes in case I wanted to use them **.

# Making Dataframe of tmdb.movies.csv

In [9]:
## create dataframe of tmdb.movies.csv
df_movies = pd.read_csv('./data/tmdb.movies.csv', index_col=0)
df_movies.head()

,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


In [10]:
df_movies.shape

(26517, 9)

# Making dataframe of rt.movie_info.tsv

In [11]:
df_info = pd.read_csv('./data/rt.movie_info.tsv', delimiter='\t')

In [12]:
df_info.head()

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN


# Making dataframe of tn.movie_budgets.csv

In [13]:
df_budget = pd.read_csv('./data/tn.movie_budgets.csv', index_col=0)

In [14]:
df_budget.head()

,release_date,movie,production_budget,domestic_gross,worldwide_gross
id,,,,,
1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [15]:
df_budget.shape

(5782, 5)

# Making dataframe of bom.movie_gross.csv

In [16]:
df_gross = pd.read_csv('./data/bom.movie_gross.csv')


In [17]:
df_gross.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [18]:
df_gross.shape

(3387, 5)